In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd
import pickle
import threading
import sys
import datetime
import glob

In [4]:
myDate = []
myText = []
for filename in glob.glob("./statements/*.txt"):
    with open(filename,'r') as file:
        myText.append(file.read().replace('\n', ' '))
        myDate.append(datetime.date(int(filename[13:17]),int(filename[17:19]),int(filename[19:21])))

In [5]:
def find_statement_startend(text):
    start = -1
    end = -1
    for i in range(len(text)):
        if text[i].find("For release")>=0:
            start = i+1
        if text[i].find("Implementation Note issued")>=0:
            end = i-1
    return [start,end]

In [6]:
def extract(url):
    statement_socket = urlopen(url)
    htmlpage = BeautifulSoup(statement_socket, 'html.parser')
    paragraphs = htmlpage.findAll('p')
    text = [paragraph.get_text().strip() for paragraph in paragraphs]
    start, end = find_statement_startend(text)
    start = max(start,0)
    if end < 0:
        end = len(text)-1
    text = text[start:end]
    statement = "".join(text)
    
    return statement

In [7]:
release_date = [datetime.date(2016,11,2),
               datetime.date(2016,12,14),
               datetime.date(2017,2,1),
               datetime.date(2017,3,15),
               datetime.date(2017,5,3),
               datetime.date(2017,6,14),
               datetime.date(2017,7,26),
               datetime.date(2017,9,20),
               datetime.date(2017,11,1),
               datetime.date(2017,12,13),
               datetime.date(2018,1,31),
               datetime.date(2018,3,21),
               datetime.date(2018,5,2),
               datetime.date(2018,6,13),
               datetime.date(2018,8,1),
               datetime.date(2018,9,26),
               datetime.date(2018,11,8),
               datetime.date(2018,12,19),
               datetime.date(2019,1,30),
               datetime.date(2019,3,20),
               datetime.date(2019,5,1),
               datetime.date(2019,6,19),
               datetime.date(2019,7,31),
               datetime.date(2019,9,18)]

In [8]:
for date in release_date:
    myDate.append(date) 
    datein = date.strftime("%Y%m%d")
    url = f"https://www.federalreserve.gov/newsevents/pressreleases/monetary{datein}a.htm"
    statement_text = extract(url)
    myText.append(statement_text)

In [9]:
df = pd.DataFrame({'Date': myDate, 'Statement': myText})
df.sort_values(by='Date', inplace=True)

In [10]:
df.head()

,Date,Statement
25,1994-02-04,Chairman Alan Greenspan announced today that t...
80,1994-03-22,Chairman Alan Greenspan announced today that t...
67,1994-05-17,The Federal Reserve today announced two action...
133,1994-08-16,The Federal Reserve announced today the follow...
129,1994-11-15,The Federal Reserve Board today approved an in...


In [11]:
df.to_csv ('fomc_statement.csv', index = False)